In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
# movies.head(2)
# movies.shape
# credits.head()
movies = movies.merge(credits,on='title')
# print(movies)
movies.head()
# budget
# homepage
# id
# original_language
# original_title
# popularity
# production_comapny
# production_countries
# release-date(not sure)
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
# movies.head()
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
# movies.head()
movies['keywords'] = movies['keywords'].apply(convert)
# movies.head()
# import ast
# b=ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')
# # print(b)
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L
movies['cast'] = movies['cast'].apply(convert)
# movies.head()
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L
movies['crew'] = movies['crew'].apply(fetch_director)
movies['overview'] = movies['overview'].apply(lambda x:x.split())
# movies.sample(5)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)
# movies.head()
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
# movies.head()
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
# new.to_csv('tags1.csv')
#new.head()
new['tags'] = new['tags'].apply(lambda x:" ".join(x))
# new.head()
# new.to_csv('tags.csv')

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()
similarity = cosine_similarity(vector)

c:\Program Files\Python310\lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [7]:

def recommend(movie):
  print(f"Movie Selected is: {movie}", end = "\n\n")
  index = new[new['title'] == movie].index[0]
  enumerated_simil = list(enumerate(similarity[index]))
  sorted_enumerated_simil = sorted(enumerated_simil, reverse=True, key=lambda x: x[1])
  print(f"Similar movies are: ")
  for movie_id, simil_score in sorted_enumerated_simil[1:6]:
    print(" ", new.iloc[movie_id].title)

In [8]:
from ipywidgets import interact 

interact(recommend, movie=new['title'])

interactive(children=(Dropdown(description='movie', options=('Avatar', "Pirates of the Caribbean: At World's E…

<function __main__.recommend(movie)>